In [1]:
!pip install pymysql

In [2]:
# IMPORTAÇÃO BIBLIOTECA
import pymysql
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
# CRIAR CONEXÃO AO MYSQL - AWS

try:
  conexao = pymysql.connect( user= 'gabriel_miranda',
                             host= 'ada-santander-1007.censored.us-east-1.rds.amazonaws.com',
                             port= 3306,
                             password= '123456',
                             database= 'gabriel_miranda_teste')
except pymysql.Error as e:
  print('Erro ao se conectar: ', e)

In [5]:
# CRIAR O CURSOR

cursor = conexao.cursor()

In [6]:
#DELETANDO TABELAS
drop_exists_produto = 'DROP TABLE IF EXISTS produto'
drop_exists_categoria = 'DROP TABLE IF EXISTS categoria'

cursor.execute(drop_exists_produto)
cursor.execute(drop_exists_categoria)


0

In [7]:
# CATEGORIA (ID_CATEGORIA, CATEGORIA)
# PRODUTO (ID_PRODUTO, ID_CATEGORIA, PRODUTO)
drop_exists = 'DROP TABLE IF EXISTS categoria'

ddl_categoria = """ CREATE TABLE categoria (
                  id_categoria int auto_increment primary key,
                  categoria varchar(200)
                 ) """

In [8]:
# EXECUÇÃO DE COMANDO SQL
cursor.execute(ddl_categoria)

0

In [9]:
# COMANDO DDL PRODUTO

ddl_produto = """  CREATE TABLE produto (
                id_produto int auto_increment primary key,
                id_categoria int,
                produto varchar(200) not null,
                foreign key (id_categoria) references categoria (id_categoria)
              ) """


In [10]:
# EXECUÇÃO DE COMANDO SQL
cursor.execute(ddl_produto)

0

In [11]:
# INSERIR REGISTROS DE CATEGORIA
dml_insert_categoria = """ INSERT INTO categoria (categoria) VALUES
                            ('Limpeza'),
                            ('Fruta')
"""

In [12]:
# EXECUTAR COMANDO DML
cursor.execute(dml_insert_categoria)

2

In [13]:
#COMMIT
conexao.commit()

In [14]:
# INSERIR REGISTROS DE PRODUTO
dml_insert_produto = """ INSERT INTO produto (id_categoria, produto) VALUES
                          (1, 'Sabão em pó'),
                          (1, 'Papel higiênico'),
                          (null, 'Sorvete')

"""

In [15]:
# EXECUTAR COMANDO SQL
cursor.execute(dml_insert_produto)

3

In [16]:
# COMMIT
conexao.commit()

In [17]:
cursor.close()

In [18]:
# Reabrindo a conexao

cursor = conexao.cursor()

Vamos **inserir**, **alterar** e **recuperar** dados do BD.

In [38]:
def inserir_categoria():
  nova_categoria = input("Categoria:")
  sql_insert_categoria = f"insert into categoria (categoria) values ('{nova_categoria}')"
  cursor.execute(sql_insert_categoria)
  conexao.commit()

  df_pesquisa = pd.read_sql_query("select * from categoria", conexao)
  print(df_pesquisa.to_markdown(index=False))


def inserir_produto():
  novo_produto = input("Produto:")

  print("Categorias disponíveis:")
  df_pesquisa = pd.read_sql_query("select * from categoria", conexao)
  print(df_pesquisa.to_markdown(index=False))

  categoria_novo_produto = int(input("Categoria (0 para nenhuma categoria):"))
  if categoria_novo_produto == 0:
    sql_insert_prouto = f"insert into produto (produto) values ('{novo_produto}')"
  else:
    sql_insert_prouto = f"insert into produto (produto, id_categoria) values ('{novo_produto}', {categoria_novo_produto})"

  cursor.execute(sql_insert_prouto)
  conexao.commit()

  df_pesquisa = pd.read_sql_query("select * from produto p left join categoria c on c.id_categoria = p.id_categoria", conexao)
  print(df_pesquisa.to_markdown(index=False))


def alterar_categoria():
  df_pesquisa = pd.read_sql_query("select * from categoria", conexao)
  print(df_pesquisa.to_markdown(index=False))

  id_categoria = int(input("Categoria a ser editada:"))

  nova_categoria = input("Novo valor de categoria:")

  sql_alterar_categoria = f"""update categoria
                              set categoria = '{nova_categoria}'
                              where id_categoria = {id_categoria}"""

  cursor.execute(sql_alterar_categoria)
  conexao.commit()

  df_pesquisa = pd.read_sql_query("select * from categoria", conexao)
  print(df_pesquisa.to_markdown(index=False))


def alterar_produto():
  df_produto = pd.read_sql_query(""" select id_produto, categoria, produto
                                    from produto p
                                    left join categoria c on c.id_categoria = p.id_categoria """, conexao)
  print(df_produto.to_markdown(index=False))

  # selecionando o produto a ser editado e recebendo  novo valor da descrição do produto
  id_produto = input('Digite o ID do produto a ser editado: ')
  valor_novo_produto = input('Qual o novo valor para DESCRIÇÃO DO PRODUTO: ')


  # selecionando a categoria que o produto pertence
  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  print(df_categoria.to_markdown(index=False))

  valor_novo_id_categoria = input('Digite o ID da Categoria do produto '+valor_novo_produto+' (use 0 para nenhuma): ')
  if int(valor_novo_id_categoria) == 0:
    sql_alterar_produto = f""" update produto
                              set id_categoria = null, produto = '{valor_novo_produto}'
                              where id_produto = {int(id_produto)} """
  else:
    sql_alterar_produto = f""" update produto
                              set id_categoria = {int(valor_novo_id_categoria)},
                                    produto = '{valor_novo_produto}'
                              where id_produto = {int(id_produto)} """


  cursor.execute(sql_alterar_produto)
  conexao.commit()

  df_produto = pd.read_sql_query(""" select id_produto, categoria, produto
                                    from produto p
                                    left join categoria c on c.id_categoria = p.id_categoria """, conexao)
  print(df_produto.to_markdown(index=False))

In [39]:
opcao_menu = int(input("""
Menu:
1 - Cadastrar ddl_categoria
2 - Cadastrar produto
3 - Alterar categoria
4 - Alterar produto
5 - Sair
"""))

if opcao_menu == 5:
  print("Sistema encerrado.")
elif opcao_menu == 1:
  inserir_categoria()
elif opcao_menu == 2:
  inserir_produto()
elif opcao_menu == 3:
  alterar_categoria()
elif opcao_menu == 4:
  alterar_produto()


Menu:
1 - Cadastrar ddl_categoria
2 - Cadastrar produto
3 - Alterar categoria
4 - Alterar produto
5 - Sair
4
|   id_produto | categoria   | produto         |
|-------------:|:------------|:----------------|
|            1 | Limpeza     | Sabão em pó     |
|            2 | Limpeza     | Papel higiênico |
|            3 |             | Sorvete         |
|            4 | padaria     | pao             |
|            5 |             | cebola          |
|            6 |             | cenoura         |
Digite o ID do produto a ser editado: 5
Qual o novo valor para DESCRIÇÃO DO PRODUTO: sabonete
|   id_categoria | categoria   |
|---------------:|:------------|
|              1 | Limpeza     |
|              2 | vegetais    |
|              3 | padaria     |
Digite o ID da Categoria do produto sabonete (use 0 para nenhuma): 1
|   id_produto | categoria   | produto         |
|-------------:|:------------|:----------------|
|            1 | Limpeza     | Sabão em pó     |
|            2 | Limpez